In [0]:
# Databricks notebook source
# ==========================================================
# UDV - CATALOGO_EQUIPOS
# Proyecto: Liga 1 Perú
# Autor: Oscar García Del Águila
# ==========================================================

from env_setup import *
from pyspark.sql import SparkSession
from utils_liga1 import setup_adls, get_dbutils, get_abfss_path, read_parquet_adls, is_dataframe_empty,get_predecesor, get_pipeline_params,get_yaml_from_param,write_delta_udv
from m_catalogo_equipos import select_final
import traceback

# ----------------------------------------------------------
# CONFIGURACIÓN DE ADLS Y SPARK
# ----------------------------------------------------------
spark = SparkSession.builder.getOrCreate()
setup_adls()
dbutils = get_dbutils()

In [0]:
# ----------------------------------------------------------
# LECTURA PRECESORES Y PARAMETROS AZURE SQL DATABASE
# ----------------------------------------------------------
entity_name="m_catalogo_equipos"

dbutils.widgets.text("prm_pipelineid", "")
prm_pipelineid = dbutils.widgets.get("prm_pipelineid")


dict_predecesores = get_predecesor(prm_pipelineid)
dict_params = get_pipeline_params(prm_pipelineid)

# ----------------------------------------------------------
# CAPTURA DE PREDECESORES
# ----------------------------------------------------------
prm_ruta_predecesor=dict_predecesores['Ruta_Predecesor']
ruta_abfss_origen = get_abfss_path(prm_ruta_predecesor)

# ----------------------------------------------------------
# CAPTURA DE PARAMETROS BD
# ----------------------------------------------------------
prm_filesystem=dict_params["FILESYSTEM"]
prm_capa_udv=dict_params["CAPA_UDV"]
prm_ruta_base=dict_params["RUTA_BASE"]
prm_ruta_tabla=dict_params["RUTA_TABLA"]
prm_formato=dict_params["FORMATO_SALIDA"]
prm_schema_tb=dict_params["SCHEMA_TABLA"]
prm_schema_vw=dict_params["SCHEMA_VISTA"]
prm_tabla_output=dict_params["NOMBRE_TABLA"]
prm_ruta_yaml=dict_params["YAML_PATH"]

prm_ruta_tabla_output=f"{prm_capa_udv}/{prm_ruta_base}/{prm_ruta_tabla}"
ruta_delta_udv = get_abfss_path(prm_ruta_tabla_output)
# ----------------------------------------------------------
# CAPTURA DE PARAMETROS YML
# ----------------------------------------------------------
yaml_conf = get_yaml_from_param(prm_ruta_yaml)

prm_cols= yaml_conf[entity_name]["cols"]
prm_schema= yaml_conf[entity_name]["schema"]
prm_columns_sql= yaml_conf[entity_name]["columns_sql"]
prm_table_comment= yaml_conf[entity_name]["table_comment"]

In [0]:
# ----------------------------------------------------------
# EJECUCIÓN SEGURA
# ----------------------------------------------------------
try:
    print("===== LECTURA DESDE RAW/DATA =====")
    df = read_parquet_adls(spark, ruta_abfss_origen)

    # Validar si el DataFrame está vacío antes de procesar
    if is_dataframe_empty(df):
        raise Exception(f"No se encontró data en la ruta origen: {ruta_abfss_origen}")

    print("===== PROCESAMIENTO EN UDV =====")
    df_final=select_final(df,prm_cols,prm_schema)

    print("===== ESCRITURA EN UDV =====")
    write_delta_udv(
    spark,
    df_final,
    schema=prm_schema_tb,
    table_name=prm_tabla_output,
    abfss_path=ruta_delta_udv,
    formato=prm_formato,
    mode="overwrite",
    #replace_where="periododia = 20251110",
    columns_sql=prm_columns_sql,
    table_comment=prm_table_comment
    )
    print(f"[SUCCESS] Proceso completado correctamente para {entity_name}")
    dbutils.notebook.exit("OK")


except Exception as e:
    print("===============================================")
    print("ERROR EN EJECUCIÓN:")
    print(str(e))
    print(traceback.format_exc())
    print("===============================================")
    dbutils.notebook.exit("ERROR")


print("===============================================")
print(f"FINALIZADO UDV - {entity_name}")
print("===============================================")
dbutils.notebook.exit("OK")